# TAIL RISK FUND: OPTIONS (BHANSALI STRATEGY)

author: [@sparshsah](https://github.com/sparshsah)

ref: https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/tail-risk-fund_options.ipynb

In [1]:
from __future__ import annotations

import dataclasses
import datetime
from typing import Final, Literal

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# https://github.com/sparshsah/foggy-statslib
import sys; sys.path.append("../../../foggy-statslib/")
import foggy_statslib.core as fsc
import foggy_statslib.fin as fsf

In [7]:
BUDGET_DOL: Final[float] = 100
START_DATE: Final[datetime.datetime] = datetime.datetime(2014, 7, 1)
ROLL_BUFFER_DAYS: Final[int] = 5  # look to roll this many trading days before expiration
IMPL_LAG_DAYS: Final[int] = 3  # trading lag in days
PREVAILING_INTEREST_RATE: Final[float] = 0.05


@dataclasses.dataclass(kw_only=True, frozen=True)
class StrategyConfig:
    """By default, buy COF puts quarterly at 20% OTM, selling iff value hits 2x initial price."""
    underlying_ticker: str = "COF"
    period_freq: Literal["M", "Q", "2Q", "Y"] = "Q"
    strike_otm_px: float = 0.20
    sell_thresh: float = 2.0

    @property
    def periods(self) -> pd.PeriodIndex:
        return pd.period_range(
            start=START_DATE,
            end=datetime.datetime.now(),
            freq=self.period_freq,
            name="period",
        # first period is burn-in
        )[1:]

    @property
    def timeseries(self) -> pd.DataFrame:
        ts = pd.DataFrame(index=self.periods)
        ts.loc[:, "entry_date"] = ts.index.start_time - ROLL_BUFFER_DAYS * pd.offsets.BDay()
        ts.loc[:, "expiration_date"] = ts.index.end_time
        return ts


class MarketDataAccess:
    """Module for getting data."""

    @staticmethod
    def get_stock_history(ticker: str = "COF") -> pd.DataFrame:
        p = (
            pd.read_csv(
                f"data/yahoo-finance_NYSE-{ticker}_px.csv",
                index_col="Date",
                parse_dates=True,
            )
            .loc[START_DATE:, "Close"]
            .rename_axis("date").rename("unadj_close_price_dol")
            .asfreq("D").ffill()
        )
        p_b = p.asfreq("B")
        r_b = p_b / p_b.shift() - 1
        vol_b = _calc_vol(r=r_b)
        vol = vol_b.asfreq("D").ffill()
        history = pd.concat([p, vol], axis="columns", verify_integrity=True)
        return history


# HELPERS


def _calc_vol(r: pd.Series[float], freq: str = "B") -> pd.Series[float]:
    """Hackily calculate realized vol from returns."""
    if r.index.freq != "B":
        raise ValueError(freq)
    return (
        (
            r
            # assert that ER_daily is close to zero
            .abs()
            # hackily remove influence of stock splits
            .clip(upper=0.20)
            **2
        )
        # a business quarter
        .rolling(window=65, min_periods=1)
        .mean()
        **0.5
        # annualize
        * 261**0.5
    ).rename("vol")

In [10]:
# DATA


def _generate_put_histories() -> pd.DataFrame:
    """Manually Black-Scholes just the data points we need given the strategy we're trying to backtest."""
    cfg = StrategyConfig()
    ts = cfg.timeseries
    sh = MarketDataAccess.get_stock_history(ticker=cfg.underlying_ticker)

    def _gen_cell(contract_ticker: pd.Period, t: pd.Timestamp) -> float:
        period = contract_ticker
        entry_date = ts.loc[period, "entry_date"]
        expiration_date = ts.loc[period, "expiration_date"]
        if entry_date <= t <= expiration_date:
            return fsf.calc_bsf_option_value(
                # set at entry
                put=True,
                K=sh.loc[entry_date, "unadj_close_price_dol"] * (1 - cfg.strike_otm_px),
                # calculated today
                S_t=sh.loc[t, "unadj_close_price_dol"],
                sigma=sh.loc[t, "vol"],  # let's assume a 1.1x vol premium
                tau=(expiration_date - t).days / 365,
                r=PREVAILING_INTEREST_RATE,
            )
        else:
            return float("nan")

    hp = pd.DataFrame(
        columns=ts.index,
        index=pd.date_range(start=START_DATE, end=sh.last_valid_index(), freq="D", name="date"),
        dtype=float,
    )
    hp = pd.DataFrame(
        {
            period: {
                t: _gen_cell(contract_ticker=period, t=t)
                for t in hp.index
            }
            for period in hp.columns
        },
        index=hp.index,
        columns=hp.columns,
    )
    return hp
PUT_HISTORIES = _generate_put_histories()
del _generate_put_histories

In [ ]:
cfg = StrategyConfig()
cfg.timeseries.head()a

In [ ]:
hs = MarketDataAccess.get_history_for_stock()
hs.head()

In [ ]:
x["vol"].plot()